In [28]:
import itertools
import pandas as pd
import numpy as np

In [6]:
file_path = r"F:\AI\hoho_python_bigdata_ml\data\marketing.csv"
data_raw = pd.read_csv(file_path)

In [7]:
data_raw

,user_id,date_served,marketing_channel,variant,converted,language_displayed,language_preferred,age_group,date_subscribed,date_canceled,subscribing_channel,is_retained
0,a100000029,1/1/18,House Ads,personalization,True,English,English,0-18 years,1/1/18,NaN,House Ads,True
1,a100000030,1/1/18,House Ads,personalization,True,English,English,19-24 years,1/1/18,NaN,House Ads,True
2,a100000031,1/1/18,House Ads,personalization,True,English,English,24-30 years,1/1/18,NaN,House Ads,True
3,a100000032,1/1/18,House Ads,personalization,True,English,English,30-36 years,1/1/18,NaN,House Ads,True
4,a100000033,1/1/18,House Ads,personalization,True,English,English,36-45 years,1/1/18,NaN,House Ads,True
...,...,...,...,...,...,...,...,...,...,...,...,...
10032,a100007290,1/17/18,Email,control,True,German,German,45-55 years,1/17/18,1/24/18,Email,False
10033,a100007291,1/17/18,Email,control,True,German,German,55+ years,1/17/18,NaN,Email,True
10034,a100007292,1/5/18,Instagram,control,False,German,German,55+ years,1/17/18,NaN,Email,True
10035,a100007293,1/17/18,Email,control,True,German,German,0-18 years,1/17/18,NaN,Email,True


In [8]:
###  extracting the needed field
columns = ['user_id', 'date_served', 'marketing_channel', 'converted']
data = data_raw[columns].copy()

### dropping null values
data.dropna(axis=0, inplace=True)

### relabel conversion to 1/0
data['converted'] = data['converted'].astype('int') 
### converting date_served into date format
data['date_served'] = pd.to_datetime(data['date_served'], format='%m/%d/%y', errors='coerce')

In [9]:
data

,user_id,date_served,marketing_channel,converted
0,a100000029,2018-01-01,House Ads,1
1,a100000030,2018-01-01,House Ads,1
2,a100000031,2018-01-01,House Ads,1
3,a100000032,2018-01-01,House Ads,1
4,a100000033,2018-01-01,House Ads,1
...,...,...,...,...
10032,a100007290,2018-01-17,Email,1
10033,a100007291,2018-01-17,Email,1
10034,a100007292,2018-01-05,Instagram,0
10035,a100007293,2018-01-17,Email,1


In [10]:
data_lvl1 = data[["user_id", "marketing_channel", "converted"]].sort_values(by = ["user_id", "marketing_channel"])
data_lvl1

,user_id,marketing_channel,converted
2380,a100000001,Facebook,0
2381,a100000002,Instagram,0
2382,a100000003,Instagram,0
1815,a100000004,House Ads,0
2383,a100000005,Facebook,0
...,...,...,...
10032,a100007290,Email,1
10033,a100007291,Email,1
10034,a100007292,Instagram,0
10035,a100007293,Email,1


In [12]:
data_lvl2 = data_lvl1.groupby(["user_id"], as_index = False).agg({"marketing_channel": lambda x: ",".join(map(str, x.unique())), "converted": max})
data_lvl2.rename(columns = {"marketing_channel": "marketing_channel_subset"}, inplace = True)

In [13]:
data_lvl2

,user_id,marketing_channel_subset,converted
0,a100000001,Facebook,0
1,a100000002,Instagram,0
2,a100000003,Instagram,0
3,a100000004,House Ads,0
4,a100000005,Facebook,0
...,...,...,...
7289,a100007290,Email,1
7290,a100007291,Email,1
7291,a100007292,Instagram,0
7292,a100007293,Email,1


In [24]:
data_lvl2.sample(10)

,user_id,marketing_channel_subset,converted
4893,a100004894,Facebook,0
430,a100000431,"Facebook,House Ads",0
1116,a100001117,Instagram,1
4668,a100004669,"House Ads,Instagram",0
1598,a100001599,Facebook,0
4163,a100004164,Instagram,0
5670,a100005671,"Facebook,House Ads",0
999,a100001000,"House Ads,Instagram",0
3995,a100003996,Push,0
5758,a100005759,House Ads,0


In [14]:
data_lvl3 = data_lvl2.groupby(["marketing_channel_subset"], as_index = False).agg(sum)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_772520\2562478285.py:1: FutureWarning: The operation <built-in function sum> failed on a column. If any error is raised, this will raise an exception in a future version of pandas. Drop these columns to avoid this warning.
  data_lvl3 = data_lvl2.groupby(["marketing_channel_subset"], as_index = False).agg(sum)


In [15]:
data_lvl3

,marketing_channel_subset,converted
0,Email,110
1,"Email,Facebook",11
2,"Email,Facebook,House Ads",8
3,"Email,Facebook,House Ads,Instagram",0
4,"Email,House Ads",40
5,"Email,House Ads,Instagram",3
6,"Email,House Ads,Push",1
7,"Email,Instagram",10
8,"Email,Push",4
9,Facebook,103


In [25]:
### return all possible combination of the channel
# 即生成列表List的所有组合
# 如List = [1,2,3]，则返回[], [1], [2], [3], [1,2], [1,3], [2,3], [1,2,3]
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in itertools.combinations(List, i+1)]
    return PS

In [30]:
def subsets(s):
    if len(s) == 1:
        return s
    else:
        sub_channels = []
        for i in range(1, len(s) + 1):
            sub_channels.extend(map(list, itertools.combinations(s, i)))
    return list(map(",".join, map(sorted, sub_channels)))

[1, 2]
[1, 3]
[2, 3]
